# Prerequisites and configuration

In [2]:
:version
:linker
:cache 500
:offline 0

0.16.0
linker: system
cache: 500 MiB
Offline mode: false


In [3]:
:dep reqwest = { features = ["blocking", "json"] }

In [4]:
:dep serde = { features = ["derive"] }
:dep serde_json

In [5]:
:dep uuid = { features = ["v4"] }

In [6]:
:dep regex

In [7]:
:dep mail-parser
:dep chrono

In [8]:
let uri = "http://127.0.0.1:41184";
let token = "c1d483c5562c0714c5145743cc35eda37487e938cc4dfeb7c985b006537b16e9bfb6192c2e8cacf562467c7147fa3ffb33499b167290bf73c54632a217c25001";

# Basic ping

In [ ]:
let page = reqwest::blocking::get(format!("{uri}/ping"));
let body = page?.text()?;
println!("{body}");

error sending request for url (http://127.0.0.1:41184/ping): error trying to connect: tcp connect error: Connection refused (os error 111)


# Async ping

In [ ]:
//:dep tokio = {features = ["full"]}
//let page = reqwest::get(format!("{uri}/ping")).await?.text().await?;

# Generic (de)pagination

In [10]:
use serde::{Deserialize, Serialize};
use serde_json::json;

#[derive(Debug, Serialize, Deserialize)]
struct Page {
    has_more: bool,
    items: Vec<serde_json::Value>,
}

fn get_pages(uri: &str) -> reqwest::Result<Vec<serde_json::Value>> {
    //println!("Fetching {uri}");
    let mut page: Page = reqwest::blocking::get(uri)?.json()?;
    let mut items = page.items;

    let mut page_num = 1;

    while page.has_more {
        let req = format!("{uri}&page={page_num}");
        //println!("Fetching {req}");
        page = reqwest::blocking::get(req)?.json()?;
        
        items.append(&mut page.items);
        page_num += 1;
        assert!(page_num < 250);
    }

    Ok(items)
}

# Initial access using token

In [ ]:
use serde::{Deserialize, Serialize};

let page = reqwest::blocking::get(format!("{uri}/notes?token={token}"))?;
let body = page.text()?;

#[derive(Debug, Serialize, Deserialize)]
struct Note {
    title: Option<String>,
    id: String,
    parent_id: String,
}

#[derive(Debug, Serialize, Deserialize)]
struct Notes {
    has_more: bool,
    items: Vec<Note>,
}

// Use the generic Value type to help write the structures above for each type...
let json: serde_json::Value = serde_json::from_str(&body)?;
println!("{json:?}");
//println!("{}", serde_json::to_string_pretty(&json)?);

let data: Notes = serde_json::from_str(&body)?;
println!("{body}");
println!("{data:?}");


error sending request for url (http://127.0.0.1:41184/notes?token=c1d483c5562c0714c5145743cc35eda37487e938cc4dfeb7c985b006537b16e9bfb6192c2e8cacf562467c7147fa3ffb33499b167290bf73c54632a217c25001): error trying to connect: tcp connect error: Connection refused (os error 111)


# Implementing `joplin search`

In [17]:
#[derive(Debug, Serialize, Deserialize)]
struct Note {
    id: String,
    title: String,
    body: String,
}

let query = "08f2e7a7c52f4725b8a4dfbf62407146%20iscompleted:0%20notebook:TODO%20type:todo";
let json = get_pages(&format!("{uri}/search?token={token}&query={query}&fields=id,title,body"))?;
let mut data: Vec<Note> = serde_json::from_value(json!(json))?;
data.sort_by(|a, b| a.id.cmp(&b.id));
data.dedup_by(|a, b| a.id == b.id);
data.len()

3

# Implementing `joplin grep`

In [ ]:
use serde_json::json;

#[derive(Debug, Serialize, Deserialize)]
struct NoteBody {
    id: String,
    body: String,
}

let json = get_pages(&format!("{uri}/notes?fields=id,body&token={token}"))?;
let mut data: Vec<NoteBody> = serde_json::from_value(json!(json))?;
data.sort_by(|a, b| a.id.cmp(&b.id));
data.dedup_by(|a, b| a.id == b.id);
println!("{}", data.len());
for body in data {
    println!("{:?}", body);
}

921
NoteBody { id: "0028c67e647a42b688363acd55a98fd8", body: "# freenode #live 2018\n\n## So, now what?\n\nLeslie Hawthorn.\n\nThis is Mr. Ben Elton: an Australian and British comedian and a prolific\nnovelist.\n\nHis habit of writing novels of dystopia is useful for this talk...\n\nAI allows machines to make the same bad decisions that humans have\ndone... faster.\n\nIf you are waiting for the time to get out and do something then that\ntime is now.\n\nSo, now what?\n\n5 Projects... [apart from freenode ;-) ]\n\n### growstuff.org\n\nSeed swaps, etc.\n\n### osbeehives.com\n\nOpen source behives. Maker project to download plans to \"print\"\nbeehives.\n\n### signal.org\n\nSignal is an encrypted text messaging app.\n\n### Mastodon\n\njoinmastodon.org\n\nOpen source alternative to twitter... that is starting to benefit from\nnetwork effects.\n\n### Better Blocker\n\niOS adware blocker.\n\n### Wrap up\n\nOur problems are not technical, they are social. We must value a free\nand just world 

()

error sending request for url (http://127.0.0.1:41184/search?token=c1d483c5562c0714c5145743cc35eda37487e938cc4dfeb7c985b006537b16e9bfb6192c2e8cacf562467c7147fa3ffb33499b167290bf73c54632a217c25001&query=xdg-mime&fields=id,title,body): error trying to connect: tcp connect error: Connection refused (os error 111)


error sending request for url (http://127.0.0.1:41184/notes?fields=id,body&token=c1d483c5562c0714c5145743cc35eda37487e938cc4dfeb7c985b006537b16e9bfb6192c2e8cacf562467c7147fa3ffb33499b167290bf73c54632a217c25001): error trying to connect: tcp connect error: Connection refused (os error 111)


# Search notebooks

In [ ]:
use serde_json::json;

#[derive(Debug, Serialize, Deserialize)]
struct Folder {
    id: String,
    parent_id: String,
    title: String,
}

let fulluri = format!("{uri}/folders?token={token}");
//let json: serde_json::Value = reqwest::blocking::get(fulluri)?.json()?;
let json = get_pages(&fulluri)?;
let mut data: Vec<Folder> = serde_json::from_value(json!(json))?;
data.sort_by(|a, b| a.title.cmp(&b.title));

data.binary_search_by(|haystack| haystack.title.as_str().cmp("TODO")).ok().map(|i| data[i].id.as_str())

error sending request for url (http://127.0.0.1:41184/folders?token=c1d483c5562c0714c5145743cc35eda37487e938cc4dfeb7c985b006537b16e9bfb6192c2e8cacf562467c7147fa3ffb33499b167290bf73c54632a217c25001): error trying to connect: tcp connect error: Connection refused (os error 111)


# Creating a note

In [ ]:
#[derive(Debug, Serialize, Deserialize)]
struct UncreatedNote {
    parent_id: String,
    title: String,
    is_todo: i32,
    body: String,
}

let note = UncreatedNote{
    parent_id: "74640eb8e8264b2ea5343619dbc5c904".to_string(),
    title: "Example".to_string(),
    is_todo: 1,
    body: "Example body".to_string(),
};

let client = reqwest::blocking::Client::new();
let res = client.post(format!("{uri}/notes?token={token}"))
    .json(&note)
    .send();
res?.status()

error sending request for url (http://127.0.0.1:41184/notes?token=c1d483c5562c0714c5145743cc35eda37487e938cc4dfeb7c985b006537b16e9bfb6192c2e8cacf562467c7147fa3ffb33499b167290bf73c54632a217c25001): error trying to connect: tcp connect error: Connection refused (os error 111)


# Attachments

## Quick scan

In [16]:
let fulluri = format!("{uri}/resources?token={token}&fields=id,title,filename,mime");
let json = get_pages(&fulluri)?;
for j in json {
    println!("{j:?}");
}

Object {"filename": String(""), "id": String("ef02442bec25432a9cf69b2f9f6cff13"), "mime": String("message/rfc822"), "title": String("JEMHC and attachments.eml")}
Object {"filename": String(""), "id": String("d39bcc7a96234ce58a8c523a9e593497"), "mime": String("image/png"), "title": String("WebClipper.png")}
Object {"filename": String(""), "id": String("9e0fbf7be00f4573ab8759f19eaeef98"), "mime": String("image/png"), "title": String("AllClients.png")}
Object {"filename": String(""), "id": String("bbd7e273a53e45ef9c14fbfc5aa2e6ef"), "mime": String("image/png"), "title": String("SubNotebooks.png")}
Object {"filename": String(""), "id": String("0c87e485afdb49dab4a16f2ad7f9cf9b"), "mime": String("image/svg+xml"), "title": String("Donate-PayPal-green_ffb7d08871de47139fd69ed302f2c877.svg")}
Object {"filename": String(""), "id": String("e9e9153a48b84e929c2014bb61721dda"), "mime": String("image/svg+xml"), "title": String("Donate-IBAN_690fde5fe86e483a89097c8a7ea5081d.svg")}
Object {"filename": St

()

## Fetch as file

In [ ]:
let id = "ef02442bec25432a9cf69b2f9f6cff13";
let page = reqwest::blocking::get(format!("{uri}/resources/{id}/file?token={token}"))?;
# TODO: How to handling binary data?
let body = page.text()?;
body

Error: expected one of `!` or `[`, found `TODO`

## Push as file

In [ ]:
#[derive(Debug, Serialize, Deserialize)]
struct UncreatedResource {
    id: Option<String>,
    title: String,
}

let resource = UncreatedResource {
    id: None,
    title: "example.txt".to_string(),
};

let form = reqwest::blocking::multipart::Form::new()
    .text("props", serde_json::to_string(&resource)?)
    .file("data", "/home/drt/Pictures/Android_greenrobot.png")?;

let client = reqwest::blocking::Client::new();
let resp = client
    .post(format!("{uri}/resources?token={token}"))
    .multipart(form)
    .send()?;
resp


Response { url: Url { scheme: "http", cannot_be_a_base: false, username: "", password: None, host: Some(Ipv4(127.0.0.1)), port: Some(41184), path: "/resources", query: Some("token=c1d483c5562c0714c5145743cc35eda37487e938cc4dfeb7c985b006537b16e9bfb6192c2e8cacf562467c7147fa3ffb33499b167290bf73c54632a217c25001"), fragment: None }, status: 200, headers: {"content-type": "application/json", "access-control-allow-origin": "*", "access-control-allow-methods": "GET, POST, OPTIONS, PUT, PATCH, DELETE", "access-control-allow-headers": "X-Requested-With,content-type", "date": "Mon, 27 Nov 2023 07:32:48 GMT", "connection": "keep-alive", "keep-alive": "timeout=5", "transfer-encoding": "chunked"} }

# Tricks

Run `cargo doc` to generate full offline documentation (VS Code will automatically prefer this to online docs).

In [ ]:
let id = uuid::Uuid::new_v4();
id.simple().to_string()

"4851c69321a64cf1818ea4ff446d86b9"

## Regex example

In [11]:
use regex::Regex;
//let markdown_code_block_re = Regex::new(r"(?m)^ {0,3}[`~]{3} *(\S+) *([\S ]+)").unwrap();
let re = r"(?m)^ {0,3}[`~]{3} *(\S+) +([\S ]+)$";
let markdown_code_block_re = Regex::new(re).unwrap();

let hay = "
rabbit         54 true
groundhog 2 true
does not match
fox   109    false
~~~sh tag
Boom
~~~

~~~sh line-2_a helo
Secondary
... boom
~~~
";

let mut fields: Vec<(&str, &str)> = vec![];
for caps in markdown_code_block_re.captures_iter(hay) {
    let (_, [language, tag]) = caps.extract();
    dbg!(cap);
    fields.push((language, tag));
}
println!("{:?}", fields);

[("sh", "tag"), ("sh", "line-2_a helo")]


[src/lib.rs:125] cap = Captures(
    {
        0: 75..84/"~~~sh tag",
        1: 78..80/"sh",
        2: 81..84/"tag",
    },
)
[src/lib.rs:125] cap = Captures(
    {
        0: 95..114/"~~~sh line-2_a helo",
        1: 98..100/"sh",
        2: 101..114/"line-2_a helo",
    },
)


In [36]:
use regex::Regex;
//let markdown_code_block_re = Regex::new(r"(?m)^ {0,3}[`~]{3} *(\S+) *([\S ]+)").unwrap();
let re = r"(?m)^ {0,3}[`~]{3} *(\S+) +([\S ]+)$";
let markdown_code_block_re = Regex::new(re).unwrap();

let hay = "
rabbit         54 true
groundhog 2 true
does not match
fox   109    false
~~~sh tag
Boom
~~~

~~~sh line-2_a helo
Secondary
... boom
~~~
";

let chunks: Vec<_> = hay.split("\n~~~").filter(|s| s.starts_with("sh ") && s.contains("\n")).map(|s| &s[3..]).collect();
let mut actions = std::collections::HashMap::new();
for &chunk in chunks.iter() {
    let mut i = chunk.splitn(2, "\n");
    // unwrap() will never fail (because we verified each chunk contains a newline)
    actions.insert(i.next().unwrap(), .i.next(().unwrap());
}
dbg!(actions);

[src/lib.rs:134] actions = {
    "line-2_a helo": "Secondary\n... boom",
    "tag": "Boom",
}
[src/lib.rs:139] caps = Captures(
    {
        0: 75..84/"~~~sh tag",
        1: 78..80/"sh",
        2: 81..84/"tag",
    },
)
[src/lib.rs:139] caps = Captures(
    {
        0: 95..114/"~~~sh line-2_a helo",
        1: 98..100/"sh",
        2: 101..114/"line-2_a helo",
    },
)


[("sh", "tag"), ("sh", "line-2_a helo")]


In [ ]:
let raw_msg = std::fs::read_to_string("mbox")?;

fn to_mbox_date(dt: &mail_parser::DateTime) -> String {
    let rfc822 = dt.to_rfc822();

    // shuffle the rfc822 into asctime format
    if dt.day >= 10 {
        format!("{} {} {} {} {}", &rfc822[0..3], &rfc822[8..11], &rfc822[5..7], &rfc822[17..25], &rfc822[12..16])
    } else {
        format!("{} {} {} {} {}", &rfc822[0..3], &rfc822[7..10], &rfc822[4..6], &rfc822[16..24], &rfc822[11..15])
    }
}
fn get_from_email<'x>(from: &'x mail_parser::Address) -> Option<&'x str>
{
        match from {
                mail_parser::Address::List(from) => {
                        Some(from[0].address()?)
                },
                _ => None,
        }
}

fn to_mbox_header(msg: &mail_parser::Message) -> Option<String> {
        let date = to_mbox_date(msg.date()?);
        let email = get_from_email(msg.from()?);
        Some(format!("From {} {}", email?, date))
}


let mut header = String::new();
        if let Some(msg) = mail_parser::MessageParser::default().parse(&raw_msg) {
        header = to_mbox_header(&msg).unwrap();
}

println!("{header}");



From support@linaro.org Thu Nov 30 12:58:00 2023


In [3]:
:dep chrono
use chrono::prelude::*;

In [4]:
FixedOffset::east_opt(if

Error: `fn(i32) -> Option<chrono::FixedOffset> {chrono::FixedOffset::west_opt}` doesn't implement `Debug`

In [3]:
use time::*;

In [6]:
Date::from_calendar_date(2023, 12.into(), 6)

Error: the trait bound `Month: From<u8>` is not satisfied